## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "prepilot"
study_type = "program"
mode_of_interest = "E-bike"

In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
# Do not run this notebook at all unless it is for a program; nbclient will run up through this cell
if study_type != "program":
    raise Exception("The plots in this notebook are only relevant to programs")

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r df_ei
%store -r dic_re
%store -r dic_pur
%store -r dic_fuel

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_fuel = defaultdict(lambda: 'Other',dic_fuel)

## Collect Data From Database

In [ ]:
expanded_ct, file_suffix, quality_text = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            dic_re,
                                                                            dic_pur=dic_pur)
expanded_ct = scaffolding.add_energy_labels(expanded_ct, df_ei, dic_fuel)

## Energy Impacts Relative to Specific Mode

In [ ]:
data_eb = expanded_ct.query(f"Mode_confirm == '{mode_of_interest}'")

In [ ]:
quality_text_mode = scaffolding.get_quality_text(expanded_ct, data_eb)

### Energy impact (kWH)

In [ ]:
#eirp : energy impact replaced_mode
eirc=expanded_ct.groupby('Replaced_mode').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
eirc.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
eirc = eirc.reset_index()
eirc = eirc.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)
eirc['boolean'] = eirc['Sketch of Total Energy_Impact(kWH)'] > 0

subset1 = eirc [['Replaced_mode', 'Sketch of Total Energy_Impact(kWH)']].copy()
subset1.rename(columns = {'Replaced_mode':'Transport Mode','Sketch of Total Energy_Impact(kWH)':'Replaced_Mode' }, inplace=True)

#eimc : energy impact mode_confirm
eimc=expanded_ct.groupby('Mode_confirm').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
eimc.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
eimc = eimc.reset_index()
eimc = eimc.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)

subset2 = eimc [['Mode_confirm', 'Sketch of Total Energy_Impact(kWH)']].copy()
subset2.rename(columns = {'Mode_confirm':'Transport Mode','Sketch of Total Energy_Impact(kWH)':'Mode_Confirm' }, inplace=True)

df_plot = pd.merge(subset1, subset2, on="Transport Mode")
df = pd.melt(df_plot , id_vars=['Transport Mode'], value_vars=['Replaced_Mode','Mode_Confirm'], var_name='selection')
df.rename(columns = {'value':'Energy Impact (kWH)'}, inplace = True)

df= df.sort_values(by=['Energy Impact (kWH)'], ascending=False)
x= 'Energy Impact (kWH)'
y= 'Transport Mode'
color = 'selection'
plot_title="Sketch of Energy Impact (kWH) by Transport Mode\n%s" % quality_text
file_name ='sketch_all_energy_impact%s' % file_suffix
overeall_energy_impact(x,y,color,df,plot_title,file_name)
alt_text = store_alt_text_bar(pd.DataFrame(df['Energy Impact (kWH)'].values, df['selection'] + '; ' + df['Transport Mode']), file_name, plot_title)

### Sketch of total energy_impact (kWH) by Replaced_mode

In [ ]:
net_energy_saved = round(sum(eirc['Sketch of Total Energy_Impact(kWH)']), 2)

x = eirc['Sketch of Total Energy_Impact(kWH)']
y = eirc['Replaced_mode']
color =eirc['boolean']

plot_title="Sketch of Energy Impact for all confirmed trips \n Contribution by mode towards a total of %s (kWH) \n%s" % (net_energy_saved, quality_text)
file_name ='sketch_all_mode_energy_impact%s' % file_suffix
energy_impact(x,y,color,plot_title,file_name)
alt_text = store_alt_text_bar(pd.DataFrame(x,y), file_name, plot_title)

### Sketch of energy impact by trips for specified mode

In [ ]:
# ebei : ebike energy impact
ebei=data_eb.groupby('Replaced_mode').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
ebei.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
ebei= ebei.reset_index()
ebei = ebei.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)
ebei['boolean'] = ebei['Sketch of Total Energy_Impact(kWH)'] > 0
net_energy_saved = round(sum(ebei['Sketch of Total Energy_Impact(kWH)']), 2)

x = ebei['Sketch of Total Energy_Impact(kWH)']
y = ebei['Replaced_mode']
color =ebei['boolean']

plot_title=f"Sketch of Energy Impact of {mode_of_interest} trips\n Contribution by replaced mode towards a total of %s (kWH)\n %s" % (net_energy_saved, quality_text_mode)
file_name ='sketch_energy_impact_mode%s' % file_suffix
energy_impact(x,y,color,plot_title,file_name)
alt_text = store_alt_text_bar(pd.DataFrame(x.values,y), file_name, plot_title)

### Sketch of total CO2 emissions by Replaced_mode

In [ ]:
co2rp=expanded_ct.groupby('Replaced_mode').agg({'CO2_Impact(lb)': ['sum', 'mean']},)
co2rp.columns = ['total_lb_CO2_emissions', 'average_lb_CO2_emission']
co2rp = co2rp.reset_index()
co2rp= co2rp.sort_values(by=['total_lb_CO2_emissions'], ascending=False)
co2rp['boolean'] = co2rp['total_lb_CO2_emissions'] > 0

net_CO2_saved = round(sum(co2rp['total_lb_CO2_emissions']), 2)

x = co2rp['total_lb_CO2_emissions']
y = co2rp['Replaced_mode']
color =co2rp['boolean']

plot_title="Sketch of CO2 Emissions Impact for all confirmed trips \n Contribution by mode towards a total of %s (kWH) \n%s" % (net_CO2_saved, quality_text)
file_name ='sketch_all_mode_CO2impact%s' % file_suffix
CO2_impact(x,y,color,plot_title,file_name)
alt_text = store_alt_text_bar(pd.DataFrame(x.values,y), file_name, plot_title)

### Sketch of CO2 emissions impact by trips of specified mode

In [ ]:
ebco2=data_eb.groupby('Replaced_mode').agg({'CO2_Impact(lb)': ['sum', 'mean']},)
ebco2.columns = ['total_lb_CO2_emissions', 'average_lb_CO2_emission']
ebco2 = ebco2.reset_index()
ebco2 = ebco2.sort_values(by=['total_lb_CO2_emissions'], ascending=False)
ebco2['boolean'] = ebco2['total_lb_CO2_emissions'] > 0
net_CO2_emissions = round(sum(ebco2['total_lb_CO2_emissions']), 2)

x = ebco2['total_lb_CO2_emissions']
y = ebco2['Replaced_mode']
color = ebco2['boolean']

plot_title=f"Sketch of Total Pounds of CO2 Emissions of {mode_of_interest} trips\n Contribution by replaced mode towards a total of %s (lb CO2 Emissions )\n %s" % (net_CO2_emissions, quality_text_mode)
file_name ='sketch_CO2impact_mode%s' % file_suffix
CO2_impact(x,y,color,plot_title,file_name)
alt_text = store_alt_text_bar(pd.DataFrame(x.values,y), file_name, plot_title)